## Learning to rank

### Problem overview

### Some literature

### Some tools

### Dataset

Dataset can be taken from hdfs:
    
    hdfs dfs -get /projects/kraken/html/l2r_dataset.csv
Or from google drive:
    
    wget https://drive.google.com/open?id=1Iy7CauA5ptOaVjskUbxvV6gbzcaDpFDK
    

In [1]:
import pandas as pd
import numpy as np

In [3]:
data=pd.read_csv('../../MLR-rozwiązania-zadan-rekrutacyjnych/dataset.csv')

In [4]:
data.shape

(4799591, 55)

In [5]:
data.head()

,listing_id,doc_rank,clicked,n0,n1,n2,n3,n4,n5,n6,...,n42,n43,n44,n45,n46,c0,c1,c2,c3,c4
0,0,0,0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,0,4,1,1
1,0,1,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,0,4,1,1
2,0,2,0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,0,4,1,1
3,0,3,0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,0,4,1,1
4,0,4,0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,0,4,1,1


Clicked column determines whever the document was relevant. Feature vector consist of Numerical values n0-n46 and categorical c0-c1. 

In [7]:
data.c0=data.c0.astype("category")
data.c1=data.c1.astype("category")
data.c2=data.c2.astype("category")
data.c3=data.c3.astype("category")
data.c4=data.c4.astype("category")

In [8]:
data.describe(include={'category'})

,c0,c1,c2,c3,c4
count,4799591,4799591,4799591,4799591,4799591
unique,4,2,9,4,3
top,0,0,4,0,1
freq,4449099,4424105,4670574,2253602,2314987


In [9]:
data.iloc[:,3:50].describe()

,n0,n1,n2,n3,n4,n5,n6,n7,n8,n9,...,n37,n38,n39,n40,n41,n42,n43,n44,n45,n46
count,4.799591e+06,4.799591e+06,4.799591e+06,4.799591e+06,4.799591e+06,4.799591e+06,4799591.0,4.799591e+06,4.799591e+06,4.799591e+06,...,4.799591e+06,4.799591e+06,4.799591e+06,4.799591e+06,4.799591e+06,4.799591e+06,4.799591e+06,4.799591e+06,4.799591e+06,4.799591e+06
mean,9.904567e-01,9.444721e-02,4.833635e-01,3.089555e-02,1.043943e-02,9.052382e-01,0.0,1.296544e+02,3.085340e+03,3.172359e+03,...,4.647375e-01,1.160406e+03,1.306972e+03,6.087273e+00,6.200157e+00,1.424516e-01,7.056783e-01,7.029861e-01,7.104559e-01,7.102847e-01
std,9.722263e-02,2.924499e-01,4.997232e-01,1.730347e-01,1.016388e-01,2.928857e-01,0.0,2.905372e+03,1.571113e+04,1.578918e+04,...,4.987551e-01,7.132471e+03,8.350907e+03,5.754844e+01,5.909137e+01,3.480947e-01,1.724934e+00,1.717792e+00,1.735944e+00,1.735536e+00
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.0,2.999000e+01,4.000000e+00,5.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.0,6.130000e+01,2.300000e+01,4.000000e+01,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,1.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.0,1.180000e+02,1.540000e+02,3.000000e+02,...,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
max,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.0,1.200000e+06,1.000000e+05,1.000000e+05,...,1.000000e+00,1.915800e+05,2.286910e+05,3.326000e+03,3.490000e+03,1.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00


The maximum number of documents in listing is:

In [10]:
data.groupby('listing_id')['clicked'].count().max()

75

and the mininmum is:

In [11]:
data.groupby('listing_id')['clicked'].count().min()

1

Number of relevant documents:

In [12]:
len(data[data.clicked==1]['listing_id'].unique())

99997

In [14]:
pd.unique(data.clicked) #binary relevance

array([0, 1])

Number of listings:

In [15]:
len(data['listing_id'].unique())

100000

## NDCG

### Task no 1. implement NDCG function:

   hint: https://en.wikipedia.org/wiki/Discounted_cumulative_gain<a href="https://en.wikipedia.org/wiki/Discounted_cumulative_gain"></a>

In [16]:
# input is an ordered collection of relevancies.
# e.g. [0,1,0] represents a listing with 3 elements, where second element is relevent (relevancy == 1)

def dcg(values):
    return 
    
def ndcg(values):   
    return dcg_ / idcg

In [26]:
# test suite
assert dcg([1,0,0]) == 1
assert dcg([0,0,1]) == 0.5
assert dcg([1,0,1]) == 1.5

assert ndcg([1,0,0]) == 1
assert ndcg([0,0,1]) == 0.5
assert ndcg([1,0,1]) == 1.5/1.6309297535714575

In [39]:
dcg([0,0,0,0,0,0,0,0,0,0,1])

0.27894294565112987

# modelling

In [28]:
listing_ids = data['listing_id'].unique()
from sklearn.model_selection import train_test_split
LISTINGS_LIMIT=1000
X_train_listings, X_test_listings = train_test_split(listing_ids[:LISTINGS_LIMIT], test_size=0.2)

In [29]:
train = data[data['listing_id'].isin(X_train_listings)]
test = data[data['listing_id'].isin(X_test_listings)]

## Task 2: calculate ndcg for every listing in test dataset , and calculate mean

In [ ]:
# expected = 0.30153690810291001